In [2]:
from tasks import run_md5
from tasks import run_fastqc
from invoke import run
import time
from datetime import datetime
from core.utils import Tibanna
from core import ff_utils

wf_md5 = "md5"
wf_fastqc = "fastqc-0-11-4-1"

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
run_md_qc = raw_input("Do you wanna run md5 and/or fastqc if missing? (md5/qc/all/none)")

# status for completion
# there are two flavors of complete signals, before it was output_file_transfer_finished, not it is complete.
# old completed wf runs have former one.

ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

################
##ADD TO WORKFLOW
# wfr_time = datetime.strptime(wfr_data['date_created'],'%Y-%m-%dT%H:%M:%S.%f+00:00')
# run_hours = int((datetime.now()-wfr_time).total_seconds()/3600)
################

def get_files(exp_set_id):
    files = []
    exps = ff_utils.get_metadata(exp_set_id, connection=ff)['experiments_in_set']
    for an_exp in exps:
        exp_resp = ff_utils.get_metadata(an_exp, connection=ff)['files']
        files.extend(exp_resp)
    return files

def summarize_file(file_resp):
    md5 = False
    qc = False
    file_id = file_resp['accession']
    sequencer = file_resp.get('instrument')
    relations = file_resp.get('related_files')
    status = file_resp.get('status')
    workflows = file_resp.get('workflow_run_inputs')
    first_alias = file_resp.get('aliases',[None])[0]
    pair_no = file_resp.get('paired_end')
    # is md5 fine
    if file_resp.get('content_md5sum'):
        md5 = True
    # is there md5sum for gzip
    if not file_resp.get('md5sum'):
        print file_id,"does not have the md5sum calculated during upload"
        
    # is there a qc?
    if file_resp.get('quality_metric'):
        qc = True
    # Check workflows for qc fastqc workflow partA
    md5_status = 'did_not_run'
    fastqc_status = 'did_not_run'
    # Assumes workflow_runs come in time ordered list, and grabs the last ones for each wf run
    if workflows:
        for a_wfr in workflows:
            wfr_resp = ff_utils.get_metadata(a_wfr['uuid'], connection=ff)
            wfr_name = wfr_resp['display_title']
            if wfr_name.startswith(wf_md5):
                md5_status = wfr_resp.get('run_status')     
            elif wfr_name.startswith(wf_fastqc):
                fastqc_status = wfr_resp.get('run_status') 
            
                
    # Check for md5 and fastqc, and if not complete, run or report it. 
    # if exclude miseq is on, do this only if sequencer is not miseq
    if not md5 or status in ["uploading", "upload failed"] or md5_status != 'complete':
        # if not, shall we run it?
        if run_md_qc in ['md5', 'all']:
            print 'md5 running for', file_resp['accession']
            code_md5= "invoke run_md5 " + env + " " + file_resp['accession'] + " " + file_resp['uuid']
            run(code_md5)
            print ''
            time.sleep(3)
        # user does not want it to be run, so just report
        else:
            print 'md5 run missing for', file_resp['accession']
    # check fastqc if md5 is fine
    else:
        if not qc or fastqc_status != 'complete':
            # if not, shall we run it?
            if run_md_qc in ['qc', 'all']:
                print 'fastqc running for', file_resp['accession']
                code_qc= "invoke run_fastqc " + env + " " + file_resp['accession'] + " " + file_resp['uuid']
                run(code_qc)
                print ''    
                time.sleep(3)
            # user does not want it to be run, so just report
            else:
                print 'fastqc run missing for', file_resp['accession'], fastqc_status
                print 
    return



all_files = ff_utils.get_metadata('files-fastq', connection=ff)['@graph']
print len(all_files)

printn = 0
counter = 0
for a_file in all_files:  
    counter += 1
    if a_file['status'] == 'deleted':
        #print "Deleted File", a_file
        continue
    if counter-printn > 100:
        print counter
        printn = counter
    print(a_file['uuid'])
    file_resp = ff_utils.get_metadata(a_file['uuid'], connection=ff, frame='embedded')
    # check if file is in s3
    head_info = tibanna.s3.does_key_exist(file_resp['upload_key'], tibanna.s3.raw_file_bucket)
    if not head_info:
        print file_resp['accession'], "does not have a file in S3"
        continue

    file_info = summarize_file(file_resp)
    
 


Do you wanna run md5 and/or fastqc if missing? (md5/qc/all/none)
3589
9321f05c-1f3f-452f-a235-ef2e3a5139d7
ca9ffc59-055a-44a6-844d-80cbbf915e1e
70e43e6b-0acf-4447-baf1-19bf8afca8a0
93d579f0-fc8e-48bd-9890-89f301d95066
99e1fc46-b8a0-4b41-a6f9-0b2328a89693
2948203f-d50b-4ff3-99a8-f9a4c7c15675
feee2fa3-9de5-4f04-b733-46804a15d5e9
882b9b1a-0483-4222-9e3a-beb9990c73bf
ccb40b4e-70b1-4b00-b538-a4a34d61f7e7
a3366ffc-ef84-4983-ba54-2d969ddeaeb8
33b808fa-3bbe-4067-92d9-367a3cb2a3fa
3b960f47-1581-4506-8291-19f4deab5e87
63400132-4310-472b-a40a-c8d029ca35a6
fa2ff84e-8e6a-42a6-b092-381715f2e929
55671903-41d1-4ec1-8753-d5c644c6f57a
1054f0d7-21ca-4ad2-87a0-065ffe74b8dd
79a85c69-be92-453a-8f5f-216c5417287d
c455b0eb-29f1-4eed-802f-eed1bbd6db31
9202d865-5c15-4c82-bbd1-74af02c5370f
7ff3b344-49ab-4d11-8aaa-2c615da6a17a
63eaf021-5e98-40bf-979c-0048d3ffebaf
d8db9ec3-4dac-43c5-b4a3-ec0123cee178
1a12eb22-2d3d-4420-ac35-09734913b35c
12470738-e650-4d65-a939-fde3fad0ea58
e81b9b58-7480-48f1-9a08-c6bd88a7bb8d
1b02c

An error occurred (404) when calling the HeadObject operation: Not Found
4DNFIYVGRWKJ does not have a file in S3
6a4ea223-46b2-480d-8e7b-c768d35b6f79
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFIQUAHAGL does not have a file in S3
2015c338-50fe-4490-bb9e-ee5c991b167c
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFIWDH48RQ does not have a file in S3
d4ae6649-f588-4047-ad14-3622fb16f02d
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFIRIQJJ4L does not have a file in S3
c9a8500b-3df9-4f89-9d08-fe9de35ab2a8
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFIU4MKVH6 does not have a file in S3
4a99dad8-8648-44ba-ad64-644be4c85aaa
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFII5JIXFQ does not have a file in S3
09668772-836c-498e-83f5-5476a1d550dc
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFIX27ZKYY does not have 

7900ea86-bc80-4488-8c03-9b57d7bf6f8f
62c8b557-965f-4d18-8262-0abf64162261
56fa8e64-5974-4540-ab1c-4338394069bb
0aa0f936-9c8c-45dd-ac7a-e7156e3cdadb
d81fec1f-3d12-488b-9c1f-a29805335db0
9b6a86df-6252-433c-ac0b-236aede69045
d1d4af1d-462c-4aaa-be49-4813f38999e3
2d987454-8eac-45a5-9b36-d0e323408a62
c5399cda-1a4b-4a16-8ffe-55dd7b1e1563
22411f5b-6445-4bc8-b067-58201821bd8c
bcef9a23-c9c4-4055-84e4-e1172610e75a
6a295508-5b24-4ae6-8116-2427e2347df5
9edabbde-96a6-4019-8118-5af3af007d2c
89271a4b-89e3-40aa-9bac-d537aecab257
dfac87c3-cf12-48d1-80cc-c7bfb9ce408d
3d398003-874a-4693-9c1d-0ceb61a0a9c8
704b6862-16b7-49c1-8d9a-2b5d94506864
323b9777-1254-46d2-85bb-4ed6617cdabe
c13d9baf-528b-4f0e-9f4c-288d8bc80c69
12476f9e-e939-4a60-b5c8-6ecaf9757d64
bc6380ba-91b6-4157-9c79-d1cc5f0a527b
03c28173-4411-4bc4-8b12-c197b8267ce0
da1a54bc-b730-4fb6-aa8e-be904897387b
88f8c60a-fc65-443d-b8e1-cc401ee00d0c
90186fa5-9eab-4713-92a7-bedbb8621c57
An error occurred (404) when calling the HeadObject operation: Not Found
4D

fastqc run missing for 4DNFISLCIDMU started

0fb1e38f-edf2-4c79-9029-7be5a2109ed0
fastqc run missing for 4DNFIYTC3RKF started

9311efed-f83e-4e3c-9ff8-d4d0522f8711
fastqc run missing for 4DNFIDVEP257 started

5226de41-0c78-418c-bde9-ccca5f906943
fastqc run missing for 4DNFIMCMUAX2 started

eea090c8-7b70-4179-80bf-94680513adc6
fastqc run missing for 4DNFIURCFHTM started

41bf27b5-fab9-4631-a3c6-99546db6982c
fastqc run missing for 4DNFI78RZF3H started

1584085c-988a-492d-845a-44f5d07eca98
fastqc run missing for 4DNFI24DAINB started

a9e43d5d-b75d-427d-8cb2-28239488987f
fastqc run missing for 4DNFIJB49KS9 started

6921a235-9a8e-4eda-91a4-14c39eaf51c7
fastqc run missing for 4DNFI91YZTQN started

f4ea2ebe-3463-488b-a49c-dc3923503f1d
fastqc run missing for 4DNFIDRUF8TL started

4300b776-8687-496e-baca-1bda751f451b
fastqc run missing for 4DNFICALV8AE started

983d23b3-a7ce-4fdb-8ce7-0ddd4f5a066d
fastqc run missing for 4DNFIZDLRC6P started

0e62a343-bbfe-43e0-9722-773586c459bf
fastqc run missi

fastqc run missing for 4DNFI8AAZCYZ started

9e5be426-6ed7-4d42-86c4-0e142279de2a
fastqc run missing for 4DNFIDRHH2AW started

ce473a04-5543-4cc4-a65f-e529d35bc46b
fastqc run missing for 4DNFIDKVUE49 started

6c69cacf-f146-4529-943f-b6a17c72e705
fastqc run missing for 4DNFI2QUE6CC started

a0077020-6fa2-416f-b5ab-7281f2173bd1
fastqc run missing for 4DNFI3PBQ83H started

bef3ccd5-6e15-46f0-9d45-df716a777af0
fastqc run missing for 4DNFILM28QP5 started

0f3c0d02-06e9-4441-891f-b281b6c7ba61
fastqc run missing for 4DNFIM1MHINN started

480876a3-f1b1-47cf-93ea-7bfabc58f222
fastqc run missing for 4DNFI9WQDUPP started

0b196924-6b6c-43fb-b483-85dc69e94939
fastqc run missing for 4DNFITLU24GA started

c26239d3-6ff8-413d-8ed8-09737e03d369
fastqc run missing for 4DNFI53W6QDY started

77262e6c-2134-4123-a9ea-faa1881f9546
fastqc run missing for 4DNFIZKAGZMW started

2d01f1ba-0412-47f4-b59f-3c3159648de7
fastqc run missing for 4DNFIAZ8Y1E4 started

40817594-d867-482a-91fb-b1e379d0d3b7
fastqc run missi

...

c6b70ec2-ab84-4412-845d-bdb9908f504b
a291f427-46f5-4f97-97c2-0829685a0a39
b93b934b-70fd-43c0-829b-221bafc2ad9a
3aca4b50-5c8e-47c1-8bc7-967959da3097
def1b1e4-fd18-4c63-a113-c8e36963ec7a
7228ccbf-2eb7-485b-8366-5ef553f0d66c
3c0b0da6-e929-436c-8c86-e49f717ab5e4
5b6f5449-c57c-4da8-b8c8-fe8fe1e20e12
5d980663-b5f6-4611-84b7-40e6ca64fdde
e946e9b3-630e-47a7-9021-ccc03c6808f0
220891cb-4b88-4322-9047-15227bdacb34
9be4fa73-5600-48e0-b5cd-eedd890306b3
87f0efd7-e4cb-4e8c-9829-905535188a88
2464dcde-a47c-4fd3-a6f7-436c303c300d
cb597023-89be-4bfe-b018-e926819139af
c7869405-12c0-4c08-b24e-5fefa437c7ab
b245c416-508e-4d5a-b229-726c31394a44
2bb75d15-d786-4269-8579-c5e5c8f864ea
7d2a4469-b235-4249-9e0c-949d65439076
5286cfab-be08-4fd5-8757-acf410410dfb
f52c2984-52a2-4b58-9196-7e0072f9d62a
e2c2fcf2-0151-4fe4-b292-08886406e6aa
ca22d060-9a85-4ca7-ab39-8fcc95b681bb
8c481980-7bd6-4aa0-b4eb-2ece9dc616c1
969998db-7417-46be-bfa0-9ef1241baacf
4b6fe57a-66c3-40cd-8cab-c24200401fc3
0dd7731e-8e52-4c44-b68c-30ad1f7cad19
a

1d6aae32-c28c-4f09-badc-e8dd0c04cea1
44d7d910-ae7e-4af6-bff6-0c8d0ce20dcb
ff0377fc-1cd7-4910-87fd-6ff28d4fe276
d0ba79aa-6fa2-4db9-a127-1aba39114e98
4ffa5820-222d-4423-b253-a34a2a92615b
de48f010-08bc-42ca-aab0-d25955c2363b
5713301f-69b2-416d-98f5-d9e13a353aff
41183d31-873a-4ee5-ae63-d825745c1a81
aa9974b2-a411-4a73-b12b-604e470e096b
e3ab2e8a-ce6f-4cab-96bc-328640cc779d
6f6b7c5c-75f6-4591-919c-73252ad172cf
f724fbd3-04eb-4863-8057-ded8e46adb81
4b035b90-03fe-4ca3-9baf-ab9186b0ccf2
0ab70cc6-9251-4485-8585-a41db7013314
4DNFID4RGP8C does not have the md5sum calculated during upload
81c58537-2a08-4343-8e31-42db154eaa3d
1770ce5b-fa93-4d6c-a85f-0b97a3b03aed
8df2c19c-dda3-467c-a835-cda8cadd1b2f
ce48e04a-00ad-4626-b3d9-9e3161432e8c
8e90cd81-f2a2-4245-b849-85bbc92b1aa6
b6e8f4f1-8af6-4cda-a585-ac6eca08a36e
becd6aa9-9d35-4f1e-9398-b82a16b90cdd
201a0c16-39c9-443f-a933-5ad6557957e4
cff746bc-d758-47e5-af55-de2479ec1c53
6c2d72ca-f861-4eec-adb8-c8758ea793b7
da9aebe0-ec1c-49e0-85b5-142254dd81a9
227cf8da-13e

e5849d3e-d464-4656-9f4b-0c5f1ccfad43
2dbf5cc1-80fa-4bb9-86ea-9cfd45dba8f0
0f403604-f31f-4809-820d-2f34dee3ffd8
5bc7865b-fa79-4f8f-8b9c-99b57a21345c
062136c5-e570-4945-bd6b-f5b266a459ce
9bc76ac4-921f-434d-9edf-af116dc07ca9
6875fe2c-f8eb-4f14-958d-c0546ae4add7
5f85d857-8e6f-4d7b-96d1-5ff21841265c
a4f01910-751f-499e-a198-c40bfe238d64
1b8800cf-09d3-46d4-a9ae-8e8bb52a8b74
9831b801-2e0b-4327-ac0f-168fe1561857
a05e56ab-53ec-4592-a030-6d18fa0575da
22b90f5f-ea79-41c1-8194-687465f78e2a
1b9d485b-04a2-4c17-9401-6a6182062fc0
7cac159f-ad66-44d2-9078-f67a821cf006
ff3b6ae6-a1f0-4462-8fa5-ce9fdf4cb0ba
0ace550e-5b2c-40aa-92bf-53f573dc057f
ab89d2be-54d0-4f3e-8ea2-3f94e3772162
d6b077b6-594b-4f98-89a2-f9ebe433d4a4
94a63c4f-d6b6-4983-9ef6-9accc296a84f
e63113fb-1b71-482b-b31b-664a8f49ebf4
c580624c-884a-4a78-835b-d42f62b3a666
9b032c0e-bd95-4027-83d8-7619f2253a12
2df3fe2f-e3f2-4583-817c-6fd0a2ec35ad
84ecbee5-ed54-421d-a912-f9b5b8cba510
0311afb7-23f9-44ff-b093-ed6046ec15ab
efd5abf0-5648-4ac1-b353-b19ade6e3baa
6

9db0b14d-ad14-4e67-9512-d0517d952808
8b048271-630f-4765-9e52-f6981292eb6c
24f95757-3568-465a-a965-1fa06571c009
c6c6084b-44e2-43f9-b6bc-9d1a53efa3e5
950f7983-7036-4a26-9a11-20927ab15aba
6a244410-6b88-4e75-b71c-b417b484825f
beaef54a-7804-4c4b-9a40-cd9895dc329b
a2dc2ab6-465c-4369-9582-9ec0ec259c2a
b1730687-0ba2-46fb-b5da-1b59b5a907b5
1ef579b1-49ef-43c1-91df-4d40916646fd
da34828f-f135-4ae1-9618-6f5b16305a99
13c31c21-3e03-4aad-aa50-8882df64fbe0
dab7f708-1217-41c3-94b9-069f36d3c14b
7702bc8b-bb52-462b-8087-d28302834b6c
a21bdcbc-6001-489f-8687-5cb6c11d6963
4bfc6ccf-5de1-4c4a-bb7f-ad64f477d204
16b246f9-886e-4002-84b4-5521f82a75c7
10354604-06f6-4f69-9037-b3a1bc86490c
d072974f-950a-4781-8bbd-7fa8315b824a
080fb6ac-11a2-4812-a838-5683bc72cf44
12edc389-3662-467e-8bca-8507e322f001
8c918b87-397a-446c-8267-524632317bf4
30914927-4911-48a3-b69e-224d226d30b7
01ba625c-ddd8-4494-8b0d-90d2982a71c6
10c8ca30-8d6b-483a-a867-d2a479f11bb3
3a45b67f-16c5-42f1-99b9-b2c2af883cf1
43e926b5-24f2-493f-ad6a-5e8c046bd3d6
0

0bd0ec16-c09d-4e0b-aa69-3cf23c7f030b
d1dba9ce-c5aa-4775-a611-b195902a16e8
8e1dab7a-efb9-40e7-a391-bfb6e284b47a
426d95cd-f230-4076-bac7-913741e5f2f8
38b49d44-626f-41ac-a1ac-4a4728ac60c2
fefd72c4-e7dc-439f-938a-ccaa4ccc0aaa
bb993a7f-ff9e-4a70-ae1a-c776bf97afe9
563ce5df-7d69-4feb-b403-273fd2bc9128
34144b43-053a-4f2a-8ebd-f89939f9c2c1
eceed4e0-6b44-402f-a944-a2409c9603bb
ec23ad28-a3d5-4981-a3d1-5a121601d7fa
91c8b84b-6d98-4b5d-a7fa-c4930edf461b
9d1cd3bd-55c4-4783-ba93-13e0a20ca0ed
b4fdd2bb-bc42-4a98-b86d-352baf2c053e
88d1054a-769b-4425-bd26-9b519e4aa39a
6aad215b-6412-4d48-8230-2074f23664ab
6f448fd1-5aa2-4ae6-8287-67508e8c96dd
59022449-2d6d-4f06-92fe-04a9e3cbe719
d235f6b9-b0c4-405b-a9b3-696a83e3049a
8dd9a1c4-1e43-414f-8a8c-597250a0e6e0
c29c7909-caaf-4f3a-b2b4-2f5813485065
d9f827b1-d3c6-4139-8287-e8e0fedccc5e
01795b9f-b843-4ac6-8391-7ef2b0497782
1f93d5a8-fe3c-404c-a10d-7e76356d1d61
e11d2eb4-972f-46af-ab35-d523ada339f1
1a05025c-ec1d-4b5d-9aec-bfabfea08808
217039e0-34e5-4a86-b6f1-69756d432aaa
7